In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def simplenet(act, s = 2):
    model = Sequential()

    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=GlorotNormal(), input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    model.add(Flatten())
    model.add(Dense(100, activation='softmax'))

    return model

In [15]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = simplenet('selu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 47s 139ms/step - loss: 4.5614 - accuracy: 0.0374 - top3_acc: 0.0936 - top_k_categorical_accuracy: 0.1402 - val_loss: 4.6486 - val_accuracy: 0.0478 - val_top3_acc: 0.1151 - val_top_k_categorical_accuracy: 0.1716
Epoch 2/50
313/313 [==============================] - 42s 133ms/step - loss: 3.8753 - accuracy: 0.1060 - top3_acc: 0.2366 - top_k_categorical_accuracy: 0.3234 - val_loss: 4.3667 - val_accuracy: 0.0764 - val_top3_acc: 0.1738 - val_top_k_categorical_accuracy: 0.2437
Epoch 3/50
313/313 [==============================] - 42s 133ms/step - loss: 3.6298 - accuracy: 0.1493 - top3_acc: 0.3014 - top_k_categorical_accuracy: 0.3941 - val_loss: 3.8871 - val_accuracy: 0.1304 - val_top3_acc: 0.2729 - val_top_k_categorical_accuracy: 0.3565
Epoch 4/50
313/313 [==============================] - 42s 134ms/step - loss: 3.4106 - accuracy: 0.1815 - top3_acc: 0.3498 - top_k_categorical_accuracy: 0.4498 - val_loss: 3.7765 - val_accuracy: 0.1399 - va

In [16]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.3050665855407715, 3.8150765895843506, 3.5795936584472656, 3.381761312484741, 3.1762590408325195, 3.001593589782715, 2.8736462593078613, 2.755958318710327, 2.6491897106170654, 2.566411256790161, 2.4776484966278076, 2.4125149250030518, 2.342266321182251, 2.275954008102417, 2.2200191020965576, 2.1688947677612305, 2.1163697242736816, 2.0683488845825195, 2.0268545150756836, 1.9853689670562744, 1.9469974040985107, 1.9030333757400513, 1.8698785305023193, 1.8336890935897827, 1.8010094165802002, 1.7631160020828247, 1.731229543685913, 1.704890251159668, 1.671431303024292, 1.640040397644043, 1.614471435546875, 1.5911457538604736, 1.560863733291626, 1.5323569774627686, 1.5073715448379517, 1.4897630214691162, 1.4637115001678467, 1.4383500814437866, 1.4198790788650513, 1.4005659818649292, 1.3799453973770142, 1.3475979566574097, 1.3463716506958008, 1.318154215812683, 1.298464298248291, 1.2767282724380493, 1.2538751363754272, 1.2441409826278687, 1.2193074226379395, 1.2078039646148682], 'ac